In [ ]:
import os
import sys
import glob
import sqlite3
from pathlib import Path
import ee
import geemap
import geopandas as gpd
import pandas as pd
import xee
import xarray as xr

sys.path.append(r'C:\Users\Pooya\w\GitHub\ShiraziPooya\DroughtMonitoringIran')

from app.utils.gee import extract_points_to_csv, run_with_adaptive_scale


ee.Authenticate()
ee.Initialize(
    project = 'drought-monitoring-iran',
    opt_url = 'https://earthengine-highvolume.googleapis.com'
)

In [ ]:
DATABASE_PATH = "../database/database.db"

conn = sqlite3.connect(DATABASE_PATH)

geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)

conn.close()

In [ ]:
DATASETS = {
    # "GPM": {
    #     "image_collection_id": "NASA/GPM_L3/IMERG_MONTHLY_V07",
    #     "start_date": "1980-01-01",
    #     "end_date": "2026-01-01",
    #     "parameter": "precipitation",
    #     "multiply": 1.0,
    #     "add": 0.0,
    #     "scale": None,
    #     "unit": "mm/hr",
    #     "Cadence": "1 Month",
    #     "name": "GPM",
    #     "output_path": f"../output/GEE/Precipitation/GPM.csv",
    #     "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    # },
    # "TRMM": {
    #     "image_collection_id": "TRMM/3B43V7",
    #     "start_date": "1980-01-01",
    #     "end_date": "2026-01-01",
    #     "parameter": "precipitation",
    #     "multiply": 1.0,
    #     "add": 0.0,
    #     "scale": None,
    #     "unit": "mm/hr",
    #     "Cadence": "1 Month",
    #     "name": "TRMM",
    #     "output_path": f"../output/GEE/Precipitation/TRMM.csv",
    #     "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    # },
    # "ERA5": {
    #     "image_collection_id": "ECMWF/ERA5_LAND/MONTHLY_AGGR",
    #     "start_date": "1980-01-01",
    #     "end_date": "2026-01-01",
    #     "parameter": "total_precipitation_sum",
    #     "multiply": 1000,
    #     "add": 0.0,
    #     "scale": None,
    #     "unit": "mm/month",
    #     "Cadence": "1 Month",
    #     "name": "ERA5",
    #     "output_path": f"../output/GEE/Precipitation/ERA5.csv",
    #     "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    # },
    # "TerraClimate": {
    #     "image_collection_id": "IDAHO_EPSCOR/TERRACLIMATE",
    #     "start_date": "1980-01-01",
    #     "end_date": "2026-01-01",
    #     "parameter": "pr",
    #     "multiply": 1,
    #     "add": 0.0,
    #     "scale": None,
    #     "unit": "mm/month",
    #     "Cadence": "1 Month",
    #     "name": "TerraClimate",
    #     "output_path": f"../output/GEE/Precipitation/TerraClimate.csv",
    #     "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    # },
    # "PERSIANN_CDR-1980_1989": {
    #     "image_collection_id": "NOAA/PERSIANN-CDR",
    #     "start_date": "1980-01-01",
    #     "end_date": "1990-01-01",
    #     "parameter": "precipitation",
    #     "multiply": 1,
    #     "add": 0.0,
    #     "scale": None,
    #     "unit": "mm/month",
    #     "Cadence": "1 Month",
    #     "name": "PERSIANN_CDR",
    #     "output_path": f"../output/GEE/Precipitation/PERSIANN_CDR-1980_1989.csv",
    #     "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    # },
    # "PERSIANN_CDR-1990_1999": {
    #     "image_collection_id": "NOAA/PERSIANN-CDR",
    #     "start_date": "1990-01-01",
    #     "end_date": "2000-01-01",
    #     "parameter": "precipitation",
    #     "multiply": 1,
    #     "add": 0.0,
    #     "scale": None,
    #     "unit": "mm/month",
    #     "Cadence": "1 Month",
    #     "name": "PERSIANN_CDR",
    #     "output_path": f"../output/GEE/Precipitation/PERSIANN_CDR-1990_1999.csv",
    #     "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    # },
    "PERSIANN_CDR-2000_2009": {
        "image_collection_id": "NOAA/PERSIANN-CDR",
        "start_date": "2000-01-01",
        "end_date": "2010-01-01",
        "parameter": "precipitation",
        "multiply": 1,
        "add": 0.0,
        "scale": None,
        "unit": "mm/month",
        "Cadence": "1 Month",
        "name": "PERSIANN_CDR",
        "output_path": f"../output/GEE/Precipitation/PERSIANN_CDR-2000_2009.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    # "PERSIANN_CDR-2010_2019": {
    #     "image_collection_id": "NOAA/PERSIANN-CDR",
    #     "start_date": "2010-01-01",
    #     "end_date": "2020-01-01",
    #     "parameter": "precipitation",
    #     "multiply": 1,
    #     "add": 0.0,
    #     "scale": None,
    #     "unit": "mm/month",
    #     "Cadence": "1 Month",
    #     "name": "PERSIANN_CDR",
    #     "output_path": f"../output/GEE/Precipitation/PERSIANN_CDR-2010_2019.csv",
    #     "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    # },
    # "PERSIANN_CDR-2020_2025": {
    #     "image_collection_id": "NOAA/PERSIANN-CDR",
    #     "start_date": "2020-01-01",
    #     "end_date": "2026-01-01",
    #     "parameter": "precipitation",
    #     "multiply": 1,
    #     "add": 0.0,
    #     "scale": None,
    #     "unit": "mm/month",
    #     "Cadence": "1 Month",
    #     "name": "PERSIANN_CDR",
    #     "output_path": f"../output/GEE/Precipitation/PERSIANN_CDR-2020_2025.csv",
    #     "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    # },
    # "CHIRPS_1980_1989": {
    #     "image_collection_id": "UCSB-CHG/CHIRPS/DAILY",
    #     "start_date": "1980-01-01",
    #     "end_date": "1990-01-01",
    #     "parameter": "precipitation",
    #     "multiply": 1,
    #     "add": 0.0,
    #     "scale": None,
    #     "unit": "mm/month",
    #     "Cadence": "1 Month",
    #     "name": "CHIRPS",
    #     "output_path": f"../output/GEE/Precipitation/CHIRPS-1980_1989.csv",
    #     "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    # },
    # "CHIRPS-1990_1999": {
    #     "image_collection_id": "UCSB-CHG/CHIRPS/DAILY",
    #     "start_date": "1990-01-01",
    #     "end_date": "2000-01-01",
    #     "parameter": "precipitation",
    #     "multiply": 1,
    #     "add": 0.0,
    #     "scale": None,
    #     "unit": "mm/month",
    #     "Cadence": "1 Month",
    #     "name": "CHIRPS",
    #     "output_path": f"../output/GEE/Precipitation/CHIRPS-1990_1999.csv",
    #     "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    # },
    "CHIRPS_2000_2009": {
        "image_collection_id": "UCSB-CHG/CHIRPS/DAILY",
        "start_date": "2000-01-01",
        "end_date": "2010-01-01",
        "parameter": "precipitation",
        "multiply": 1,
        "add": 0.0,
        "scale": None,
        "unit": "mm/month",
        "Cadence": "1 Month",
        "name": "CHIRPS",
        "output_path": f"../output/GEE/Precipitation/CHIRPS-2000_2009.csv",
        "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    },
    # "CHIRPS-2010_2019": {
    #     "image_collection_id": "UCSB-CHG/CHIRPS/DAILY",
    #     "start_date": "2010-01-01",
    #     "end_date": "2020-01-01",
    #     "parameter": "precipitation",
    #     "multiply": 1,
    #     "add": 0.0,
    #     "scale": None,
    #     "unit": "mm/month",
    #     "Cadence": "1 Month",
    #     "name": "CHIRPS",
    #     "output_path": f"../output/GEE/Precipitation/CHIRPS-2010_2019.csv",
    #     "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    # },
    # "CHIRPS-2020_2025": {
    #     "image_collection_id": "UCSB-CHG/CHIRPS/DAILY",
    #     "start_date": "2020-01-01",
    #     "end_date": "2026-01-01",
    #     "parameter": "precipitation",
    #     "multiply": 1,
    #     "add": 0.0,
    #     "scale": None,
    #     "unit": "mm/month",
    #     "Cadence": "1 Month",
    #     "name": "CHIRPS",
    #     "output_path": f"../output/GEE/Precipitation/CHIRPS-2020_2025.csv",
    #     "points_geojson": "../assets/geo_data/MazandaranStationsIRIMO.geojson"
    # },
}

In [ ]:
# for name, config in DATASETS.items():
#     print(f"Product: {name}")
#     extract_points_to_csv(
#         **config,
#         points_geojson="../assets/geo_data/MazandaranStationsIRIMO.geojson",
#     )

In [ ]:
for name, config in DATASETS.items():
    print(f"Product: {name}")
    df = run_with_adaptive_scale(
        config=config,
        base_points_geojson=config["points_geojson"],
        scale_factors=[2, 3, 4, 5, 7, 8, 9, 10, 15, 20]
    )  
    out_path = Path(config["output_path"])
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_path, index=False)
    print(f"Final merged result saved to {out_path}")

# Concat Data

In [ ]:
folder = "../output/GEE/Precipitation/"

all_dfs = []

for filepath in glob.glob(os.path.join(folder, "*.csv")):
    df = pd.read_csv(filepath)
    
    filename = os.path.basename(filepath)
    model_name, _ = os.path.splitext(filename)
    
    model_name = model_name.split("-")[0]

    df = df.rename(columns={model_name: "Precipitation"})
    df["model"] = model_name

    all_dfs.append(df)

result = pd.concat(all_dfs, ignore_index=True)
result["date"] = pd.to_datetime(result["date"], format="mixed", dayfirst=True, errors="coerce")
result = result.sort_values(by=["Region", "St_Name", "model", "date"]).reset_index(drop=True)

result

# Convert Long to Wide

In [ ]:
result = result.pivot_table(
    index=["St_ID", "St_Name", "St_Lat", "St_Lon", "St_Ele", "date", "region_id"],
    columns="model",
    values="Precipitation"
).reset_index()

result

# Concat geoinfo and results

In [ ]:
result = pd.merge(
    left=result,
    right=geoinfo,
    left_on=["region_id", "St_Name"],
    right_on=["region_id", "station_name"],
    how="left"
).drop(columns=["St_ID", "St_Name", "St_Lat", "St_Lon", "St_Ele"])

cols = ["region_id", "region_name", "station_id", "station_name", "lat", "lon", "station_elevation"]
result = result[cols + [c for c in result.columns if c not in cols]]

result.sort_values(by=["region_id", "region_name", "station_id", "station_name", "date"], inplace=True)

result

In [ ]:
conn = sqlite3.connect(DATABASE_PATH)

result.to_sql('gee_precip_monthly', conn, if_exists='replace', index=False)

conn.commit()
conn.close()